In [1]:
import random
import json
from typing import List, Dict, Tuple, Any
from dataclasses import dataclass
from enum import Enum
import itertools


# UDA-Bench Query Generation System

This notebook implements an automated query generation system for UDA-Bench with 5 main categories:
1. **Select**: Extract operations only
2. **Filter**: Extract + Filter operations (6 subcategories)
3. **Join**: Extract + Join operations (binary and multi-table)
4. **Agg**: Extract + Aggregation operations
5. **Mixed**: Combination of multiple operators (32 subcategories)


First, we need define 

1. the attribute type, which could varies from str,int,float by its value.
2. the specific usage of attribute, such as categoried attributes for group by, numerical attributes for count,avg etc.
3. the source of this attribute, such as text, images ...

In [2]:
class AttributeType(Enum):
    """Attribute value types"""
    STRING = "str"
    INTEGER = "int"
    FLOAT = "float"
    BOOLEAN = "bool"


class AttributeUsage(Enum):
    """Specific usage of attribute in queries"""
    CATEGORICAL = "categorical"    # For GROUP BY, categorical filters
    NUMERICAL = "numerical"        # For COUNT, SUM, AVG, MIN, MAX, numerical comparisons
    GENERAL = "general"            # For all usage


class AttributeModality(Enum):
    """Source modality of the attribute"""
    TEXT = "text"           # Plain text data
    IMAGE = "image"         # Image data (e.g., X-ray, photos)
    TABLE = "table"         # Table data
    STRUCTURED = "structured"  # Structured data from database


@dataclass
class Attribute:
    """
    Represents a database attribute with rich metadata.
    
    Attributes:
        name: Name of the attribute (column name)
        table: Table this attribute belongs to
        value_type: Data type of the attribute values (str, int, float, bool)
        usage: How this attribute is typically used in queries
        modality: Source modality of the data (text, image, table, etc.)
        is_nullable: Whether this attribute can have NULL values
        description: Optional description of the attribute
    """
    name: str
    table: str
    value_type: AttributeType
    usage: AttributeUsage
    modality: AttributeModality
    is_nullable: bool = False
    description: str = ""
    
    @property
    def full_name(self) -> str:
        """Return fully qualified name: table.attribute"""
        return f"{self.table}.{self.name}"

    def attribute_name(self) -> str:
        """Return the name of the attribute"""
        return self.name
    
    def is_groupable(self) -> bool:
        """Check if attribute can be used in GROUP BY"""
        return self.usage in [AttributeUsage.CATEGORICAL]
    
    def is_aggregatable(self) -> bool:
        """Check if attribute can be used with aggregation functions"""
        return self.usage == AttributeUsage.NUMERICAL
    
    def is_joinable(self) -> bool:
        """Check if attribute can be used as join key"""
        return self.usage == AttributeUsage.IDENTIFIER
    
    def supports_comparison(self) -> bool:
        """Check if attribute supports comparison operators (<, >, =, etc.)"""
        return self.value_type in [AttributeType.INTEGER, AttributeType.FLOAT, AttributeType.STRING]
    
    def supports_like(self) -> bool:
        """Check if attribute supports LIKE operator"""
        return self.value_type == AttributeType.STRING



In [3]:
# load the /data/dengqiyan/UDA-Bench/Query/Art/Art_attributes.json and to form list[Attribute]

def load_attributes_from_json(json_path: str) -> List[Attribute]:
    """
    Load attributes from a JSON file and convert them to Attribute objects.
    
    Args:
        json_path: Path to the JSON file containing attribute definitions
    
    Returns:
        List of Attribute objects
    """
    # Mapping from string values to Enum types
    type_mapping = {
        "str": AttributeType.STRING,
        "int": AttributeType.INTEGER,
        "float": AttributeType.FLOAT,
        "bool": AttributeType.BOOLEAN
    }
    
    usage_mapping = {
        "categorical": AttributeUsage.CATEGORICAL,
        "numerical": AttributeUsage.NUMERICAL,
        "general": AttributeUsage.GENERAL
    }
    
    modality_mapping = {
        "text": AttributeModality.TEXT,
        "image": AttributeModality.IMAGE,
        "table": AttributeModality.TABLE,
        "structured": AttributeModality.STRUCTURED
    }
    
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    attributes = []
    
    # Handle both flat list and nested dict formats
    if isinstance(data, dict):
        # Nested format: {"Wiki_Text": [...], "Wiki_Art": [...]}
        for source_key, attr_list in data.items():
            for attr_dict in attr_list:
                attr = Attribute(
                    name=attr_dict["name"],
                    table=attr_dict["table"],
                    value_type=type_mapping.get(attr_dict["value_type"], AttributeType.STRING),
                    usage=usage_mapping.get(attr_dict["usage"], AttributeUsage.GENERAL),
                    modality=modality_mapping.get(attr_dict["modality"], AttributeModality.TEXT),
                    is_nullable=attr_dict.get("is_nullable", False),
                    description=attr_dict.get("description", "")
                )
                attributes.append(attr)
    elif isinstance(data, list):
        # Flat format: [...]
        for attr_dict in data:
            attr = Attribute(
                name=attr_dict["name"],
                table=attr_dict["table"],
                value_type=type_mapping.get(attr_dict["value_type"], AttributeType.STRING),
                usage=usage_mapping.get(attr_dict["usage"], AttributeUsage.GENERAL),
                modality=modality_mapping.get(attr_dict["modality"], AttributeModality.TEXT),
                is_nullable=attr_dict.get("is_nullable", False),
                description=attr_dict.get("description", "")
            )
            attributes.append(attr)
    
    return attributes


# Load Art attributes
attributes_path = "/data/dengqiyan/UDA-Bench/Query/Player/Player_attributes.json"
attributes: List[Attribute] = load_attributes_from_json(attributes_path)


# Group by modality
text_attrs = [a for a in attributes if a.modality == AttributeModality.TEXT]
image_attrs = [a for a in attributes if a.modality == AttributeModality.IMAGE]



## Select Templates

In this cell, it is a function that recieve the avaliable attributes, which is a list like list:class Attribute, and a table name, then  populate the placeholder with the random avaliable attributes, but if exists an atribute with AttributeModality.IMAGE, then must be concluded one image attribute. The template is as "SELECT {attribute(s)} FROM {table}.

In [4]:
def generate_select_query(attributes: List[Attribute], 
                          table: str, 
                          attribute_num: int = 1,
                          image_num: int = 1,
                          seed: int = None) -> Tuple[str, List[Attribute]]:
    """
    Generate a SELECT query by populating placeholders with random attributes.
    
    If any IMAGE modality attribute exists in the available attributes,
    at least one image attribute MUST be included in the selection.
    
    Args:
        attributes: List of available Attribute objects
        table: Table name for the FROM clause
        min_attrs: Minimum number of attributes to select (default: 1)
        max_attrs: Maximum number of attributes to select (default: all)
        seed: Random seed for reproducibility (optional)
    
    Returns:
        Tuple of (SQL query string, list of selected attributes)
    """
    if seed is not None:
        random.seed(seed)
    
    if not attributes:
        raise ValueError("Attribute list cannot be empty")
    
    
    # Separate image attributes from non-image attributes
    image_attrs = [attr for attr in attributes if attr.modality == AttributeModality.IMAGE]
    non_image_attrs = [attr for attr in attributes if attr.modality != AttributeModality.IMAGE]
    
    selected_attrs = []
    
    # If image attributes exist, must include at least one
    if image_attrs:
        # Select at least one image attribute
        # num_image_to_select = random.randint(1, min(len(image_attrs), max_attrs))
        selected_image_attrs = random.sample(image_attrs, image_num)
        selected_attrs.extend(selected_image_attrs)
    
    # Calculate remaining slots for non-image attributes
    remaining_slots = attribute_num - image_num
    
    # Fill remaining slots with non-image attributes
    if non_image_attrs and remaining_slots > 0:
        num_non_image = min(remaining_slots, len(non_image_attrs))
        selected_non_image_attrs = random.sample(non_image_attrs, num_non_image)
        selected_attrs.extend(selected_non_image_attrs)
    
    # Shuffle to randomize order
    random.shuffle(selected_attrs)
    
    # Build the SELECT clause
    attribute_names = ", ".join([attr.name for attr in selected_attrs])
    
    # Generate the SQL query
    sql_query = f"SELECT {attribute_names} FROM {table};"
    
    return sql_query, selected_attrs



# Generate a few example queries
print("Example SELECT queries:")
print("-" * 50)
for i in range(3):
    query, selected = generate_select_query(attributes, "Art", attribute_num=3, image_num=1)
    print(f"\nQuery {i+1}: {query}")
    print(f"  Selected attributes: {[attr.name for attr in selected]}")
    print(f"  Modalities: {[attr.modality.value for attr in selected]}")


Example SELECT queries:
--------------------------------------------------

Query 1: SELECT birth_date, olympic_gold_medals FROM Art;
  Selected attributes: ['birth_date', 'olympic_gold_medals']
  Modalities: ['text', 'text']

Query 2: SELECT NBA_team, nationality FROM Art;
  Selected attributes: ['NBA_team', 'nationality']
  Modalities: ['text', 'text']

Query 3: SELECT founded_year, college FROM Art;
  Selected attributes: ['founded_year', 'college']
  Modalities: ['text', 'text']


## Filter Templates

template: SELECT {attribute(s)} From {table} WHERE subcategories.

subcategory 1 : WHERE \{Attribute\}\{Operator\}\{Literal\}

subcategory 2 : WHERE two filters with AND :  \{Attribute\}\{Operator\}\{Literal\} AND \{Attribute\}\{Operator\}\{Literal\}

subcategory 3 : WHERE two filters with OR: \{Attribute\}\{Operator\}\{Literal\} OR \{Attribute\}\{Operator\}\{Literal\}

subcategory 4 : WHERE three and more filters \{Attribute\}\{Operator\}\{Literal\} (AND \{Attribute\}\{Operator\}\{Literal\})^n

subcategory 5 : WHERE three and more filters \{Attribute\}\{Operator\}\{Literal\} (OR \{Attribute\}\{Operator\}\{Literal\})^n

subcategory 6 : WHERE three and more filters with AND OR combination


we further classify it by the number of filters (one filter, two filters, and more than two filters) and the way of combining the filters  (conjunction only, disjunction only, and conjunction of disjunctions), resulting in a total of 6 \texttt{Filter} sub-categories.  With varying complexities, these sub-categories of queries offer a large space for cost optimization (e.g., more filters bring a large optimization opportunity for filter reordering). 

As shown in the example, the pattern ``\{Attribute\}\{Operator\}\{Literal\}'' serves as the placeholder format for filter predicates. During query generation, we randomly sample both the filter attribute and the comparison operator (such as $\leq$, $=$, or $\geq$) with equal probability, and we further sample literal values to produce varying selectivities. 

In [5]:
import pandas as pd
from collections import Counter

class DataStatistics:
    """
    Class to load ground truth data and compute statistics for realistic literal generation.
    """
    
    def __init__(self, csv_path: str):
        """
        Load CSV data and compute statistics for each column.
        
        Args:
            csv_path: Path to the ground truth CSV file
        """
        self.df = pd.read_csv(csv_path)
        self.total_rows = len(self.df)
        self.column_stats = {}
        self._compute_statistics()
    
    def _compute_statistics(self):
        """Compute value distribution statistics for each column."""
        # Define columns that use || as multi-value separator
        MULTI_VALUE_SEPARATOR = '||'
        
        for col in self.df.columns:
            # Skip non-filterable columns
            if col in ['id', 'Artwork_URL', 'intro_url']:
                continue
            
            values = self.df[col].dropna()
            
            if len(values) == 0:
                continue
            
            # Determine column type
            if pd.api.types.is_numeric_dtype(values):
                self.column_stats[col] = {
                    'type': 'numeric',
                    'is_multi_value': False,
                    'min': float(values.min()),
                    'max': float(values.max()),
                    'mean': float(values.mean()),
                    'median': float(values.median()),
                    'percentiles': {
                        10: float(values.quantile(0.1)),
                        25: float(values.quantile(0.25)),
                        50: float(values.quantile(0.5)),
                        75: float(values.quantile(0.75)),
                        90: float(values.quantile(0.9))
                    },
                    'values': sorted(values.unique().tolist())
                }
            else:
                # String/categorical column - check for multi-value attributes
                str_values = values.astype(str)
                
                # Check if this column contains multi-value entries (using ||)
                has_multi_value = str_values.str.contains(r'\|\|', regex=True).any()
                
                if has_multi_value:
                    # Split multi-value entries and count individual values
                    all_individual_values = []
                    for val in str_values:
                        # Split by || and strip whitespace
                        parts = [p.strip() for p in val.split(MULTI_VALUE_SEPARATOR)]
                        all_individual_values.extend(parts)
                    
                    value_counts = Counter(all_individual_values)
                else:
                    # Single value column
                    value_counts = Counter(str_values)
                
                sorted_by_freq = sorted(value_counts.items(), key=lambda x: x[1], reverse=True)
                
                self.column_stats[col] = {
                    'type': 'categorical',
                    'is_multi_value': has_multi_value,
                    'unique_count': len(value_counts),
                    'value_counts': dict(sorted_by_freq),
                    # High frequency values (common, high selectivity)
                    'high_freq_values': [v for v, c in sorted_by_freq[:10]],
                    # Medium frequency values
                    'medium_freq_values': [v for v, c in sorted_by_freq[10:30] if c > 1],
                    # Low frequency values (rare, low selectivity)
                    'low_freq_values': [v for v, c in sorted_by_freq if c <= 2][:20],
                    'all_values': list(value_counts.keys())
                }
    
    def get_literal_by_selectivity(self, attr_name: str, selectivity: str = "medium") -> Tuple[str, float]:
        """
        Get a literal value based on the desired selectivity.
        
        Args:
            attr_name: Name of the attribute (must match column name, case-insensitive)
            selectivity: 'low' (rare values, few results), 
                        'medium' (moderate frequency),
                        'high' (common values, many results)
        
        Returns:
            Tuple of (literal_value, estimated_selectivity_ratio)
        """
        # Find matching column (case-insensitive)
        col_name = None
        for col in self.column_stats.keys():
            if col.lower() == attr_name.lower():
                col_name = col
                break
        
        if col_name is None:
            # Fallback to default values
            return "'unknown'", 0.0
        
        stats = self.column_stats[col_name]
        
        if stats['type'] == 'numeric':
            return self._get_numeric_literal(stats, selectivity)
        else:
            return self._get_categorical_literal(stats, selectivity)
    
    def _get_numeric_literal(self, stats: Dict, selectivity: str) -> Tuple[str, float]:
        """Generate numeric literal based on selectivity using percentiles."""
        percentiles = stats['percentiles']
        
        if selectivity == "low":
            # Use extreme values (top 10%) - fewer results when using < or >
            value = percentiles[90]
            ratio = 0.1
        elif selectivity == "high":
            # Use values around median - more results
            value = percentiles[50]
            ratio = 0.5
        else:  # medium
            # Use 25th or 75th percentile
            value = random.choice([percentiles[25], percentiles[75]])
            ratio = 0.25
        
        # Round to integer if all values are integers
        if all(isinstance(v, int) or (isinstance(v, float) and v.is_integer()) 
               for v in stats['values'][:10]):
            return str(int(value)), ratio
        else:
            return f"{value:.2f}", ratio
    
    def _get_categorical_literal(self, stats: Dict, selectivity: str) -> Tuple[str, float]:
        """Generate categorical literal based on selectivity using frequency."""
        value_counts = stats['value_counts']
        total = sum(value_counts.values())
        
        if selectivity == "low":
            # Pick rare values (low frequency)
            candidates = stats['low_freq_values']
            if not candidates:
                candidates = stats['all_values'][-10:]  # Last 10 values
        elif selectivity == "high":
            # Pick common values (high frequency)
            candidates = stats['high_freq_values']
        else:  # medium
            candidates = stats['medium_freq_values']
            if not candidates:
                candidates = stats['all_values']
        
        if not candidates:
            candidates = stats['all_values']
        
        value = random.choice(candidates)
        count = value_counts.get(value, 1)
        ratio = count / total
        
        # Escape single quotes in value
        escaped_value = value.replace("'", "''")
        return f"'{escaped_value}'", ratio
    
    def get_column_info(self, col_name: str) -> Dict:
        """Get statistics for a specific column."""
        for col in self.column_stats.keys():
            if col.lower() == col_name.lower():
                return self.column_stats[col]
        return None


# Load ground truth data
gt_data_path = "/data/dengqiyan/UDA-Bench/Query/Player/player.csv"
data_stats = DataStatistics(gt_data_path)

# Print summary of loaded data
print(f"✅ Loaded {data_stats.total_rows} rows from Art_Groud_Truth.csv")
print(f"📊 Analyzed {len(data_stats.column_stats)} columns")

# Show multi-value columns
multi_value_cols = [col for col, stats in data_stats.column_stats.items() 
                    if stats.get('is_multi_value', False)]
# print(f"\n🔗 Multi-value columns (using || separator): {multi_value_cols}")

# print("\nColumn Statistics Summary:")
# print("-" * 70)

# for col, stats in list(data_stats.column_stats.items()):
#     if stats['type'] == 'numeric':
#         print(f"  📈 {col}: numeric, range=[{stats['min']:.0f}, {stats['max']:.0f}]")
#     else:
#         multi_flag = " [MULTI-VALUE]" if stats.get('is_multi_value') else ""
#         print(f"  📋 {col}: categorical, {stats['unique_count']} unique values{multi_flag}")
#         if stats['high_freq_values']:
#             # Show individual values (not combined with ||)
#             print(f"      Top values: {stats['high_freq_values'][:5]}")


✅ Loaded 141 rows from Art_Groud_Truth.csv
📊 Analyzed 14 columns


In [6]:
# Comparison operators for different attribute types
NUMERIC_OPERATORS = ['=', '!=', '<', '<=', '>', '>=']
STRING_OPERATORS = ['=', '!=', 'LIKE']
CATEGORICAL_OPERATORS = ['=', '!=', 'IN']


def generate_literal(attr: Attribute, selectivity: str = "medium", 
                     stats: DataStatistics = None) -> Tuple[str, float]:
    """
    Generate a literal value for a given attribute based on real data statistics.
    
    Args:
        attr: The attribute to generate a literal for
        selectivity: 'low' (rare values, fewer results), 
                    'medium' (moderate frequency),
                    'high' (common values, more results)
        stats: DataStatistics object with real data (uses global data_stats if None)
    
    Returns:
        Tuple of (literal_value_string, selectivity_ratio)
    """
    if stats is None:
        stats = data_stats
    
    # Try to get literal from real data
    literal, ratio = stats.get_literal_by_selectivity(attr.name, selectivity)
    
    if literal != "'unknown'":
        return literal, ratio
    
    # Fallback to generated values if column not found
    if attr.value_type == AttributeType.INTEGER:
        if selectivity == "low":
            return str(random.randint(80, 100)), 0.1
        elif selectivity == "high":
            return str(random.randint(0, 20)), 0.5
        else:
            return str(random.randint(30, 70)), 0.25
    
    elif attr.value_type == AttributeType.FLOAT:
        if selectivity == "low":
            return f"{random.uniform(80, 100):.2f}", 0.1
        elif selectivity == "high":
            return f"{random.uniform(0, 20):.2f}", 0.5
        else:
            return f"{random.uniform(30, 70):.2f}", 0.25
    
    elif attr.value_type == AttributeType.STRING:
        if attr.usage == AttributeUsage.CATEGORICAL:
            samples = ["'value_A'", "'value_B'", "'value_C'"]
        else:
            samples = ["'%keyword%'", "'prefix%'", "'%suffix'", "'exact_match'"]
        return random.choice(samples), 0.1
    
    elif attr.value_type == AttributeType.BOOLEAN:
        return random.choice(["0", "1"]), 0.5
    
    return "'unknown'", 0.0


def generate_predicate(attr: Attribute, selectivity: str = "medium", 
                       stats: DataStatistics = None) -> Tuple[str, float]:
    """
    Generate a single filter predicate: {Attribute}{Operator}{Literal}
    
    Args:
        attr: The attribute for the predicate
        selectivity: Selectivity level for the literal
        stats: DataStatistics object for real data
    
    Returns:
        Tuple of (predicate_string, selectivity_ratio)
    """
    if stats is None:
        stats = data_stats
    
    # Choose operator based on attribute type
    if attr.value_type in [AttributeType.INTEGER, AttributeType.FLOAT]:
        operator = random.choice(NUMERIC_OPERATORS)
    elif attr.usage == AttributeUsage.CATEGORICAL:
        operator = random.choice(CATEGORICAL_OPERATORS)
    else:
        operator = random.choice(STRING_OPERATORS)
    
    literal, ratio = generate_literal(attr, selectivity, stats)
    
    # Handle special operators
    # if operator == 'LIKE':
    #     # For LIKE, use pattern matching
    #     col_info = stats.get_column_info(attr.name)
    #     if col_info and col_info['type'] == 'categorical':
    #         # Use actual value with wildcards
    #         base_value = literal.strip("'")
    #         like_literal = f"'%{base_value[:3]}%'" if len(base_value) > 3 else f"'%{base_value}%'"
    #         return f"{attr.name} LIKE {like_literal}", ratio * 2  # LIKE usually matches more
    #     return f"{attr.name} LIKE {literal}", ratio
    
    # elif operator == 'IN':
    #     # Generate multiple values for IN clause
    #     literals_with_ratios = [generate_literal(attr, selectivity, stats) for _ in range(random.randint(2, 4))]
    #     literals = [l[0] for l in literals_with_ratios]
    #     combined_ratio = min(1.0, sum(l[1] for l in literals_with_ratios))
    #     return f"{attr.name} IN ({', '.join(literals)})", combined_ratio
    
    # else:
    #     return f"{attr.name} {operator} {literal}", ratio
    return f"{attr.name} {operator} {literal}", ratio


def generate_filter_query(
    attributes: List[Attribute],
    table: str,
    subcategory: int,
    select_attr_num: int = 2,
    image_num: int = 1,
    filter_count: int = 3,
    seed: int = None
) -> Tuple[str, Dict]:
    """
    Generate a Filter query based on the subcategory.
    
    Subcategories:
        1: Single filter - WHERE {A}{O}{L}
        2: Two filters with AND - WHERE {A}{O}{L} AND {A}{O}{L}
        3: Two filters with OR - WHERE {A}{O}{L} OR {A}{O}{L}
        4: Multiple filters with AND - WHERE {A}{O}{L} (AND {A}{O}{L})^n
        5: Multiple filters with OR - WHERE {A}{O}{L} (OR {A}{O}{L})^n
        6: Mixed AND/OR combination - WHERE (... AND ...) OR (... AND ...)
    
    Args:
        attributes: List of available Attribute objects
        table: Table name
        subcategory: Filter subcategory (1-6)
        select_attr_num: Number of attributes in SELECT clause
        image_num: Number of image attributes to include
        filter_count: Number of filters for subcategory 4, 5, 6 (default 3)
        seed: Random seed
    
    Returns:
        Tuple of (SQL query string, metadata dict)
    """
    if seed is not None:
        random.seed(seed)
    
    # Generate SELECT clause using existing function
    select_query, selected_attrs = generate_select_query(
        attributes, table, select_attr_num, image_num, seed
    )
    select_clause = select_query.replace(f" FROM {table};", "")
    
    # Get filterable attributes (exclude image attributes for filtering)
    filterable_attrs = [a for a in attributes if a.modality != AttributeModality.IMAGE]
    
    if not filterable_attrs:
        raise ValueError("No filterable attributes available")
    
    # Generate predicates based on subcategory
    metadata = {
        "subcategory": subcategory,
        "filter_combination": "",
        "num_filters": 0,
        "predicates": []
    }
    
    if subcategory == 1:
        # Single filter
        attr = random.choice(filterable_attrs)
        selectivity = random.choice(["low", "medium", "high"])
        predicate, ratio = generate_predicate(attr, selectivity)
        where_clause = predicate
        metadata["filter_combination"] = "single"
        metadata["num_filters"] = 1
        metadata["predicates"] = [predicate]
        metadata["selectivity_ratios"] = [ratio]
    
    elif subcategory == 2:
        # Two filters with AND
        attrs = random.sample(filterable_attrs, min(2, len(filterable_attrs)))
        pred_results = [generate_predicate(a, random.choice(["low", "medium", "high"])) for a in attrs]
        predicates = [p[0] for p in pred_results]
        ratios = [p[1] for p in pred_results]
        where_clause = " AND ".join(predicates)
        metadata["filter_combination"] = "AND"
        metadata["num_filters"] = 2
        metadata["predicates"] = predicates
        metadata["selectivity_ratios"] = ratios
    
    elif subcategory == 3:
        # Two filters with OR
        attrs = random.sample(filterable_attrs, min(2, len(filterable_attrs)))
        pred_results = [generate_predicate(a, random.choice(["low", "medium", "high"])) for a in attrs]
        predicates = [p[0] for p in pred_results]
        ratios = [p[1] for p in pred_results]
        where_clause = " OR ".join(predicates)
        metadata["filter_combination"] = "OR"
        metadata["num_filters"] = 2
        metadata["predicates"] = predicates
        metadata["selectivity_ratios"] = ratios
    
    elif subcategory == 4:
        # Multiple filters with AND (3 or more)
        n = max(3, filter_count)
        attrs = random.choices(filterable_attrs, k=n)
        pred_results = [generate_predicate(a, random.choice(["low", "medium", "high"])) for a in attrs]
        predicates = [p[0] for p in pred_results]
        ratios = [p[1] for p in pred_results]
        where_clause = " AND ".join(predicates)
        metadata["filter_combination"] = "AND"
        metadata["num_filters"] = n
        metadata["predicates"] = predicates
        metadata["selectivity_ratios"] = ratios
    
    elif subcategory == 5:
        # Multiple filters with OR (3 or more)
        n = max(3, filter_count)
        attrs = random.choices(filterable_attrs, k=n)
        pred_results = [generate_predicate(a, random.choice(["low", "medium", "high"])) for a in attrs]
        predicates = [p[0] for p in pred_results]
        ratios = [p[1] for p in pred_results]
        where_clause = " OR ".join(predicates)
        metadata["filter_combination"] = "OR"
        metadata["num_filters"] = n
        metadata["predicates"] = predicates
        metadata["selectivity_ratios"] = ratios
    
    elif subcategory == 6:
        # Mixed AND/OR combination: (p1 AND p2) OR (p3 AND p4) or similar CNF/DNF
        n = max(4, filter_count)
        attrs = random.choices(filterable_attrs, k=n)
        pred_results = [generate_predicate(a, random.choice(["low", "medium", "high"])) for a in attrs]
        predicates = [p[0] for p in pred_results]
        ratios = [p[1] for p in pred_results]
        
        # Create groups for mixed combination
        mid = n // 2
        group1 = " AND ".join(predicates[:mid])
        group2 = " AND ".join(predicates[mid:])
        where_clause = f"({group1}) OR ({group2})"
        metadata["filter_combination"] = "AND_OR_mixed"
        metadata["num_filters"] = n
        metadata["predicates"] = predicates
        metadata["selectivity_ratios"] = ratios
    
    else:
        raise ValueError(f"Invalid subcategory: {subcategory}. Must be 1-6.")
    
    # Build final SQL query
    sql_query = f"{select_clause} FROM {table} WHERE {where_clause};"
    
    metadata["selected_attributes"] = [a.name for a in selected_attrs]
    metadata["where_clause"] = where_clause
    
    return sql_query, metadata


# Example: Generate one query for each subcategory with REAL data values
print("=" * 70)
print("FILTER QUERY EXAMPLES (6 Subcategories) - Using Real Data")
print("=" * 70)

subcategory_names = {
    1: "Single Filter",
    2: "Two Filters (AND)",
    3: "Two Filters (OR)",
    4: "Multiple Filters (AND)",
    5: "Multiple Filters (OR)",
    6: "Mixed AND/OR Combination"
}

for subcat in range(1, 7):
    print(f"\n{'─' * 70}")
    print(f"Subcategory {subcat}: {subcategory_names[subcat]}")
    print(f"{'─' * 70}")
    
    query, meta = generate_filter_query(
        attributes, 
        table="Art", 
        subcategory=subcat,
        select_attr_num=3,
        image_num=1,
        filter_count=4
    )
    
    print(f"SQL: {query}")
    print(f"  Combination: {meta['filter_combination']}")
    print(f"  Num Filters: {meta['num_filters']}")
    print(f"  Predicates:")
    for pred, ratio in zip(meta['predicates'], meta['selectivity_ratios']):
        print(f"    • {pred}  (selectivity: {ratio:.1%})")


FILTER QUERY EXAMPLES (6 Subcategories) - Using Real Data

──────────────────────────────────────────────────────────────────────
Subcategory 1: Single Filter
──────────────────────────────────────────────────────────────────────
SQL: SELECT draft_year, team_name FROM Art WHERE mvp_awards = 0;
  Combination: single
  Num Filters: 1
  Predicates:
    • mvp_awards = 0  (selectivity: 50.0%)

──────────────────────────────────────────────────────────────────────
Subcategory 2: Two Filters (AND)
──────────────────────────────────────────────────────────────────────
SQL: SELECT birth_date, draft_year FROM Art WHERE college LIKE 'University of Florida' AND area = '%keyword%';
  Combination: AND
  Num Filters: 2
  Predicates:
    • college LIKE 'University of Florida'  (selectivity: 1.5%)
    • area = '%keyword%'  (selectivity: 10.0%)

──────────────────────────────────────────────────────────────────────
Subcategory 3: Two Filters (OR)
─────────────────────────────────────────────────────────

## Aggregate Category


\noindent\underline{(3) \textit{For the \texttt{Agg} category}}, during query generation, we randomly sample the group-by attribute, the aggregated attributes, and the aggregation functions (such as \texttt{Count}) to populate the corresponding placeholders (i.e., \{agg\_func\}(\{Attribute\}(s)). 

Template
SELECT {attribute(s)}, {Aggregation func}({numerical attribute}) FROM {table} GROUP BY {categorical attribute}.
其中, COUNT({any attribute})

In [7]:
# Aggregation functions
AGG_FUNCTIONS = ['COUNT', 'SUM', 'AVG', 'MIN', 'MAX']
# COUNT can be applied to any attribute, others require numerical attributes
NUMERIC_ONLY_AGG = ['SUM', 'AVG', 'MIN', 'MAX']


def generate_agg_query(
    attributes: List[Attribute],
    table: str,
    select_attr_num: int = 2,
    image_num: int = 1,
    num_group_by: int = 1,
    num_agg_funcs: int = 1,
    seed: int = None
) -> Tuple[str, Dict]:
    """
    Generate an Aggregation query.
    
    Template: SELECT {attribute(s)}, {agg_func}({attribute}) FROM {table} GROUP BY {categorical attribute(s)}
    
    Rules:
    - GROUP BY attributes must be categorical (usage == CATEGORICAL)
    - COUNT can be applied to any attribute
    - SUM, AVG, MIN, MAX can only be applied to numerical attributes
    - At least one image attribute should be in SELECT (if available)
    
    Args:
        attributes: List of available Attribute objects
        table: Table name
        select_attr_num: Number of regular attributes in SELECT clause (excluding aggregations)
        image_num: Number of image attributes to include
        num_group_by: Number of GROUP BY attributes
        num_agg_funcs: Number of aggregation functions to apply
        seed: Random seed
    
    Returns:
        Tuple of (SQL query string, metadata dict)
    """
    if seed is not None:
        random.seed(seed)
    
    # Separate attributes by their properties
    categorical_attrs = [a for a in attributes if a.usage == AttributeUsage.CATEGORICAL]
    numerical_attrs = [a for a in attributes if a.usage == AttributeUsage.NUMERICAL]
    image_attrs = [a for a in attributes if a.modality == AttributeModality.IMAGE]
    non_image_attrs = [a for a in attributes if a.modality != AttributeModality.IMAGE]
    
    if not categorical_attrs:
        raise ValueError("No categorical attributes available for GROUP BY")
    
    # Select GROUP BY attributes (must be categorical)
    num_group_by = min(num_group_by, len(categorical_attrs))
    group_by_attrs = random.sample(categorical_attrs, num_group_by)
    
    # Select regular attributes for SELECT clause (including at least one image if available)
    selected_attrs = []
    
    # Add image attributes if available
    if image_attrs and image_num > 0:
        num_image = min(image_num, len(image_attrs))
        selected_attrs.extend(random.sample(image_attrs, num_image))
    
    # Add non-image attributes (exclude those already in group_by)
    remaining_non_image = [a for a in non_image_attrs if a not in group_by_attrs and a not in selected_attrs]
    remaining_slots = select_attr_num - len(selected_attrs)
    
    if remaining_non_image and remaining_slots > 0:
        num_to_add = min(remaining_slots, len(remaining_non_image))
        selected_attrs.extend(random.sample(remaining_non_image, num_to_add))
    
    # Generate aggregation expressions
    agg_expressions = []
    agg_metadata = []
    
    for _ in range(num_agg_funcs):
        # Randomly choose aggregation function
        agg_func = random.choice(AGG_FUNCTIONS)
        
        if agg_func in NUMERIC_ONLY_AGG:
            # Must use numerical attribute
            if not numerical_attrs:
                agg_func = 'COUNT'  # Fallback to COUNT
                agg_attr = random.choice(non_image_attrs)
            else:
                agg_attr = random.choice(numerical_attrs)
        else:
            # COUNT can use any attribute (prefer non-image)
            agg_attr = random.choice(non_image_attrs) if non_image_attrs else random.choice(attributes)
        
        # Create aggregation expression with alias
        alias = f"{agg_func.lower()}_{agg_attr.name}"
        agg_expr = f"{agg_func}({agg_attr.name}) AS {alias}"
        agg_expressions.append(agg_expr)
        
        agg_metadata.append({
            "function": agg_func,
            "attribute": agg_attr.name,
            "alias": alias
        })
    
    # Build SELECT clause
    # Include: group_by attributes + selected attributes + aggregation expressions
    select_parts = []
    
    # Add group by attributes first
    for attr in group_by_attrs:
        select_parts.append(attr.name)
    
    # Add selected attributes (image and others)
    for attr in selected_attrs:
        if attr.name not in select_parts:
            select_parts.append(attr.name)
    
    # Add aggregation expressions
    select_parts.extend(agg_expressions)
    
    select_clause = ", ".join(select_parts)
    
    # Build GROUP BY clause
    group_by_clause = ", ".join([attr.name for attr in group_by_attrs])
    
    # Build final SQL query
    sql_query = f"SELECT {select_clause} FROM {table} GROUP BY {group_by_clause};"
    
    metadata = {
        "category": "Agg",
        "group_by_attributes": [attr.name for attr in group_by_attrs],
        "selected_attributes": [attr.name for attr in selected_attrs],
        "aggregations": agg_metadata,
        "num_group_by": num_group_by,
        "num_aggregations": num_agg_funcs
    }
    
    return sql_query, metadata


# Example: Generate aggregation queries
print("=" * 70)
print("AGGREGATION QUERY EXAMPLES")
print("=" * 70)

# Example 1: Single GROUP BY, single aggregation
print(f"\n{'─' * 70}")
print("Example 1: Single GROUP BY + COUNT")
print(f"{'─' * 70}")
query, meta = generate_agg_query(
    attributes, 
    table="Art", 
    select_attr_num=2,
    image_num=1,
    num_group_by=1,
    num_agg_funcs=1
)
print(f"SQL: {query}")
print(f"  GROUP BY: {meta['group_by_attributes']}")
print(f"  Aggregations: {meta['aggregations']}")

# Example 2: Multiple GROUP BY, multiple aggregations
print(f"\n{'─' * 70}")
print("Example 2: Multiple GROUP BY + Multiple Aggregations")
print(f"{'─' * 70}")
query, meta = generate_agg_query(
    attributes, 
    table="Art", 
    select_attr_num=2,
    image_num=1,
    num_group_by=2,
    num_agg_funcs=2
)
print(f"SQL: {query}")
print(f"  GROUP BY: {meta['group_by_attributes']}")
print(f"  Aggregations: {meta['aggregations']}")

# Example 3: With different aggregation functions
print(f"\n{'─' * 70}")
print("Example 3: Various Aggregation Functions")
print(f"{'─' * 70}")
for i in range(3):
    query, meta = generate_agg_query(
        attributes, 
        table="Player", 
        select_attr_num=1,
        image_num=1,
        num_group_by=1,
        num_agg_funcs=1,
        seed=i*10
    )
    print(f"  {i+1}. {query}")
    print(f"     → {meta['aggregations'][0]['function']}({meta['aggregations'][0]['attribute']})")


AGGREGATION QUERY EXAMPLES

──────────────────────────────────────────────────────────────────────
Example 1: Single GROUP BY + COUNT
──────────────────────────────────────────────────────────────────────
SQL: SELECT nationality, nba_championships, fiba_world_cup, MIN(fiba_world_cup) AS min_fiba_world_cup FROM Art GROUP BY nationality;
  GROUP BY: ['nationality']
  Aggregations: [{'function': 'MIN', 'attribute': 'fiba_world_cup', 'alias': 'min_fiba_world_cup'}]

──────────────────────────────────────────────────────────────────────
Example 2: Multiple GROUP BY + Multiple Aggregations
──────────────────────────────────────────────────────────────────────
SQL: SELECT position, nationality, NBA_team, age, COUNT(mvp_awards) AS count_mvp_awards, MAX(championships) AS max_championships FROM Art GROUP BY position, nationality;
  GROUP BY: ['position', 'nationality']
  Aggregations: [{'function': 'COUNT', 'attribute': 'mvp_awards', 'alias': 'count_mvp_awards'}, {'function': 'MAX', 'attribute':

## JOIN Category

\noindent\underline{(4) \textit{For the \texttt{Join} category}}, 
to thoroughly evaluate the capacities of different systems in optimizing join queries, we curate two sub-categories of join queries, i.e., binary joins and multi-table joins, where more tables incur higher computation cost. During query generation, we explicitly define a join graph w.r.t. each dataset. For example, in \med, valid join paths include Disease $\bowtie$ Drug, along with their corresponding join keys (e.g., Disease.name = Drug.disease). Similarly, in \nba, we have Players $\bowtie$ Teams $\bowtie$ Managers, together with  join keys that link these tables.

two table joins template:
SELECT {attribute(s)} FROM {table1} JOIN {table2} on {table1.join_key} = {table2.join_key}

etc.

输入，所有会用到的表的gt的地址 e.g. {player:"xxx", team:"xxx"}

In [8]:
import os
from typing import Optional

@dataclass
class JoinPath:
    """Represents a join relationship between two tables."""
    left_table: str
    right_table: str
    left_key: str
    right_key: str
    
    def to_sql_condition(self) -> str:
        """Return the SQL JOIN condition."""
        return f"{self.left_table}.{self.left_key} = {self.right_table}.{self.right_key}"


@dataclass
class TableConfig:
    """Configuration for a single table."""
    name: str
    csv_path: str
    attributes: List[Attribute]


class JoinGraph:
    """
    Manages join relationships between tables and generates JOIN queries.
    """
    
    def __init__(self, tables: Dict[str, TableConfig], join_paths: List[JoinPath]):
        """
        Initialize the join graph.
        
        Args:
            tables: Dictionary mapping table names to TableConfig objects
            join_paths: List of JoinPath objects defining valid joins
        """
        self.tables = tables
        self.join_paths = join_paths
        self.data_stats = {}  # DataStatistics for each table
        
        # Load data statistics for each table
        for table_name, config in tables.items():
            if os.path.exists(config.csv_path):
                self.data_stats[table_name] = DataStatistics(config.csv_path)
    
    def get_join_path(self, table1: str, table2: str) -> Optional[JoinPath]:
        """Find a join path between two tables."""
        for jp in self.join_paths:
            if (jp.left_table == table1 and jp.right_table == table2):
                return jp
            if (jp.left_table == table2 and jp.right_table == table1):
                # Return reversed join path
                return JoinPath(table1, table2, jp.right_key, jp.left_key)
        return None
    
    def find_multi_table_path(self, tables: List[str]) -> List[JoinPath]:
        """
        Find a sequence of join paths connecting multiple tables.
        Uses simple BFS to find connected path.
        """
        if len(tables) < 2:
            return []
        
        path = []
        remaining = set(tables[1:])
        current = tables[0]
        visited = {current}
        
        while remaining:
            found = False
            for next_table in list(remaining):
                jp = self.get_join_path(current, next_table)
                if jp:
                    path.append(jp)
                    visited.add(next_table)
                    remaining.remove(next_table)
                    current = next_table
                    found = True
                    break
            
            if not found:
                # Try to find path through visited tables
                for v in visited:
                    for next_table in list(remaining):
                        jp = self.get_join_path(v, next_table)
                        if jp:
                            path.append(jp)
                            visited.add(next_table)
                            remaining.remove(next_table)
                            found = True
                            break
                    if found:
                        break
            
            if not found:
                raise ValueError(f"Cannot find join path for tables: {remaining}")
        
        return path
    
    def get_all_attributes(self, table_names: List[str] = None) -> List[Attribute]:
        """Get all attributes from specified tables (or all tables if None)."""
        if table_names is None:
            table_names = list(self.tables.keys())
        
        all_attrs = []
        for name in table_names:
            if name in self.tables:
                all_attrs.extend(self.tables[name].attributes)
        return all_attrs


def generate_binary_join_query(
    join_graph: JoinGraph,
    table1: str,
    table2: str,
    select_attr_num: int = 3,
    seed: int = None
) -> Tuple[str, Dict]:
    """
    Generate a binary (2-table) JOIN query.
    
    Template: SELECT {attributes} FROM {table1} JOIN {table2} ON {join_condition}
    
    Args:
        join_graph: JoinGraph with table configurations
        table1: First table name
        table2: Second table name
        select_attr_num: Number of attributes to select
        seed: Random seed
    
    Returns:
        Tuple of (SQL query string, metadata dict)
    """
    if seed is not None:
        random.seed(seed)
    
    # Get join path
    join_path = join_graph.get_join_path(table1, table2)
    if not join_path:
        raise ValueError(f"No join path between {table1} and {table2}")
    
    # Get attributes from both tables
    attrs1 = join_graph.tables[table1].attributes
    attrs2 = join_graph.tables[table2].attributes
    all_attrs = attrs1 + attrs2
    
    # Select random attributes (ensure from both tables)
    selected = []
    num_from_t1 = max(1, select_attr_num // 2)
    num_from_t2 = select_attr_num - num_from_t1
    
    if attrs1:
        selected.extend(random.sample(attrs1, min(num_from_t1, len(attrs1))))
    if attrs2:
        selected.extend(random.sample(attrs2, min(num_from_t2, len(attrs2))))
    
    random.shuffle(selected)
    
    # Build SELECT clause with table prefixes
    select_parts = [f"{attr.table}.{attr.name}" for attr in selected]
    select_clause = ", ".join(select_parts)
    
    # Build SQL query
    sql_query = f"SELECT {select_clause} FROM {table1} JOIN {table2} ON {join_path.to_sql_condition()};"
    
    metadata = {
        "category": "Join",
        "subcategory": "binary_join",
        "tables": [table1, table2],
        "join_condition": join_path.to_sql_condition(),
        "selected_attributes": [f"{a.table}.{a.name}" for a in selected]
    }
    
    return sql_query, metadata


def generate_multi_table_join_query(
    join_graph: JoinGraph,
    tables: List[str],
    select_attr_num: int = 4,
    seed: int = None
) -> Tuple[str, Dict]:
    """
    Generate a multi-table (>2) JOIN query.
    
    Template: SELECT {attributes} FROM {table1} 
              JOIN {table2} ON {cond1} 
              JOIN {table3} ON {cond2} ...
    
    Args:
        join_graph: JoinGraph with table configurations
        tables: List of table names to join (minimum 3)
        select_attr_num: Number of attributes to select
        seed: Random seed
    
    Returns:
        Tuple of (SQL query string, metadata dict)
    """
    if seed is not None:
        random.seed(seed)
    
    if len(tables) < 3:
        raise ValueError("Multi-table join requires at least 3 tables")
    
    # Find join path
    join_paths = join_graph.find_multi_table_path(tables)
    
    # Get attributes from all tables
    all_attrs = []
    for table in tables:
        all_attrs.extend(join_graph.tables[table].attributes)
    
    # Select attributes (at least one from each table)
    selected = []
    per_table = max(1, select_attr_num // len(tables))
    
    for table in tables:
        table_attrs = join_graph.tables[table].attributes
        if table_attrs:
            num_select = min(per_table, len(table_attrs))
            selected.extend(random.sample(table_attrs, num_select))
    
    # Add more if needed
    remaining = [a for a in all_attrs if a not in selected]
    while len(selected) < select_attr_num and remaining:
        selected.append(random.choice(remaining))
        remaining.remove(selected[-1])
    
    random.shuffle(selected)
    
    # Build SELECT clause
    select_parts = [f"{attr.table}.{attr.name}" for attr in selected]
    select_clause = ", ".join(select_parts)
    
    # Build FROM and JOIN clauses
    from_table = tables[0]
    join_clauses = []
    
    joined_tables = {from_table}
    for jp in join_paths:
        # Determine which table to add
        if jp.right_table not in joined_tables:
            join_clauses.append(f"JOIN {jp.right_table} ON {jp.to_sql_condition()}")
            joined_tables.add(jp.right_table)
        elif jp.left_table not in joined_tables:
            # Reverse the join
            reversed_cond = f"{jp.right_table}.{jp.right_key} = {jp.left_table}.{jp.left_key}"
            join_clauses.append(f"JOIN {jp.left_table} ON {reversed_cond}")
            joined_tables.add(jp.left_table)
    
    # Build SQL query
    sql_query = f"SELECT {select_clause} FROM {from_table} " + " ".join(join_clauses) + ";"
    
    metadata = {
        "category": "Join",
        "subcategory": "multi_table_join",
        "tables": tables,
        "num_joins": len(join_paths),
        "join_conditions": [jp.to_sql_condition() for jp in join_paths],
        "selected_attributes": [f"{a.table}.{a.name}" for a in selected]
    }
    
    return sql_query, metadata


def save_queries_to_file(queries: List[Dict], output_path: str, format: str = "json"):
    """
    Save generated queries to a file.
    
    Args:
        queries: List of query dictionaries with 'sql' and 'metadata' keys
        output_path: Path to save the file
        format: Output format ('json' or 'sql')
    """
    # Ensure output directory exists
    output_dir = os.path.dirname(output_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
    
    if format == "json":
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(queries, f, indent=2, ensure_ascii=False)
        print(f"✅ Saved {len(queries)} queries to {output_path}")
        
    elif format == "sql":
        with open(output_path, 'w', encoding='utf-8') as f:
            for i, q in enumerate(queries):
                subcategory = q.get('metadata', {}).get('subcategory', 'unknown')
                tables = q.get('metadata', {}).get('tables', [])
                f.write(f"-- Query {i+1}: {subcategory} ({', '.join(tables) if tables else 'N/A'})\n")
                f.write(q['sql'] + "\n\n")
        print(f"✅ Saved {len(queries)} queries to {output_path}")
    
    else:
        raise ValueError(f"Unsupported format: {format}. Use 'json' or 'sql'.")


# =============================================================================
# Setup Player Dataset Join Graph
# =============================================================================

# Define Player dataset tables and their CSV paths
player_base_path = "/data/dengqiyan/UDA-Bench/Query/Player"

# Load attributes for each table
player_attrs_data = json.load(open(f"{player_base_path}/Player_attributes.json"))

def attrs_from_json(attr_list: List[Dict]) -> List[Attribute]:
    """Convert JSON attribute list to Attribute objects."""
    type_map = {"str": AttributeType.STRING, "int": AttributeType.INTEGER, "float": AttributeType.FLOAT}
    usage_map = {"categorical": AttributeUsage.CATEGORICAL, "numerical": AttributeUsage.NUMERICAL, "general": AttributeUsage.GENERAL}
    modality_map = {"text": AttributeModality.TEXT, "image": AttributeModality.IMAGE}
    
    return [
        Attribute(
            name=a["name"], table=a["table"],
            value_type=type_map.get(a["value_type"], AttributeType.STRING),
            usage=usage_map.get(a["usage"], AttributeUsage.GENERAL),
            modality=modality_map.get(a["modality"], AttributeModality.TEXT),
            is_nullable=a.get("is_nullable", False),
            description=a.get("description", "")
        ) for a in attr_list
    ]

# Create table configurations
player_tables = {
    "player": TableConfig("player", f"{player_base_path}/player.csv", attrs_from_json(player_attrs_data["player"])),
    "team": TableConfig("team", f"{player_base_path}/team.csv", attrs_from_json(player_attrs_data["team"])),
    "manager": TableConfig("manager", f"{player_base_path}/manager.csv", attrs_from_json(player_attrs_data["manager"])),
    "city": TableConfig("city", f"{player_base_path}/city.csv", attrs_from_json(player_attrs_data["city"])),
}

# Define join paths based on the data structure
# player.team = team.team_name
# team.location = city.city_name
# team.ownership = manager.name
# manager.nba_team = team.team_name
player_join_paths = [
    JoinPath("player", "team", "team", "team_name"),
    JoinPath("team", "city", "location", "city_name"),
    JoinPath("team", "manager", "ownership", "name"),
    JoinPath("manager", "team", "nba_team", "team_name"),
]

# Create join graph
player_join_graph = JoinGraph(player_tables, player_join_paths)

print("✅ Player Join Graph initialized")
print(f"   Tables: {list(player_tables.keys())}")
print(f"   Join Paths:")
for jp in player_join_paths:
    print(f"      {jp.left_table}.{jp.left_key} = {jp.right_table}.{jp.right_key}")


✅ Player Join Graph initialized
   Tables: ['player', 'team', 'manager', 'city']
   Join Paths:
      player.team = team.team_name
      team.location = city.city_name
      team.ownership = manager.name
      manager.nba_team = team.team_name


In [9]:
# =============================================================================
# Generate JOIN Queries and Save to Player Directory
# =============================================================================

all_join_queries = []

# -----------------------------------------------------------------------------
# 1. Binary Join Queries (2 tables)
# -----------------------------------------------------------------------------
print("=" * 70)
print("GENERATING BINARY JOIN QUERIES")
print("=" * 70)

binary_join_pairs = [
    ("player", "team"),
    ("team", "city"),
    ("team", "manager"),
]

for t1, t2 in binary_join_pairs:
    print(f"\n{'─' * 70}")
    print(f"Join: {t1} ⟕ {t2}")
    print(f"{'─' * 70}")
    
    # Generate 10 queries per join pair
    for i in range(10):
        query, meta = generate_binary_join_query(
            player_join_graph,
            t1, t2,
            select_attr_num=4,
            seed=i * 100 + hash(t1 + t2) % 1000
        )
        
        all_join_queries.append({
            "sql": query,
            "metadata": meta
        })
        
        if i < 2:  # Print first 2 examples
            print(f"  {i+1}. {query}")
    
    print(f"  ... Generated 10 queries for {t1} ⟕ {t2}")

# -----------------------------------------------------------------------------
# 2. Multi-Table Join Queries (3+ tables)
# -----------------------------------------------------------------------------
print(f"\n{'=' * 70}")
print("GENERATING MULTI-TABLE JOIN QUERIES")
print("=" * 70)

multi_join_combinations = [
    ["player", "team", "city"],
    ["player", "team", "manager"],
    ["team", "city", "manager"],
    ["player", "team", "city", "manager"],  # 4-table join
]

for tables in multi_join_combinations:
    print(f"\n{'─' * 70}")
    print(f"Join: {' ⟕ '.join(tables)}")
    print(f"{'─' * 70}")
    
    # Generate 10 queries per combination
    for i in range(10):
        try:
            query, meta = generate_multi_table_join_query(
                player_join_graph,
                tables,
                select_attr_num=5,
                seed=i * 200 + hash(str(tables)) % 1000
            )
            
            all_join_queries.append({
                "sql": query,
                "metadata": meta
            })
            
            if i < 2:  # Print first 2 examples
                print(f"  {i+1}. {query}")
        except Exception as e:
            print(f"  ⚠️ Error: {e}")
            break
    
    print(f"  ... Generated 10 queries for {' ⟕ '.join(tables)}")

# -----------------------------------------------------------------------------
# 3. Save Queries to Files
# -----------------------------------------------------------------------------
print(f"\n{'=' * 70}")
print("SAVING QUERIES")
print("=" * 70)

output_dir = "/data/dengqiyan/UDA-Bench/Query/Player"

# Save as JSON
save_queries_to_file(
    all_join_queries,
    f"{output_dir}/join_queries.json",
    format="json"
)

# Save as SQL file
save_queries_to_file(
    all_join_queries,
    f"{output_dir}/join_queries.sql",
    format="sql"
)

# Summary
print(f"\n📊 Summary:")
print(f"   Total queries generated: {len(all_join_queries)}")
binary_count = sum(1 for q in all_join_queries if q['metadata']['subcategory'] == 'binary_join')
multi_count = sum(1 for q in all_join_queries if q['metadata']['subcategory'] == 'multi_table_join')
print(f"   Binary joins: {binary_count}")
print(f"   Multi-table joins: {multi_count}")


GENERATING BINARY JOIN QUERIES

──────────────────────────────────────────────────────────────────────
Join: player ⟕ team
──────────────────────────────────────────────────────────────────────
  1. SELECT team.championships, team.location, player.age, player.olympic_gold_medals FROM player JOIN team ON player.team = team.team_name;
  2. SELECT team.founded_year, team.team_name, player.name, player.team FROM player JOIN team ON player.team = team.team_name;
  ... Generated 10 queries for player ⟕ team

──────────────────────────────────────────────────────────────────────
Join: team ⟕ city
──────────────────────────────────────────────────────────────────────
  1. SELECT city.city_name, city.population, team.founded_year, team.location FROM team JOIN city ON team.location = city.city_name;
  2. SELECT city.state_name, city.area, team.founded_year, team.championships FROM team JOIN city ON team.location = city.city_name;
  ... Generated 10 queries for team ⟕ city

──────────────────────